In [20]:
# Import the necessary modules
import pandas as pd
import numpy as np
import matplotlib as plt
from prophet import Prophet
import seaborn as sns


# My questions:
# # 3) What is the average cart value per customer?
#     ... in the Northeast?
#     ... in the South?
#     ... in the Midwest?
#     ... in the Southwest?
#     ... in the Northwest?
#     ... outside the contiguous US states?
#
# 6) What is the average city sales volume?
#     ... in the Northeast?
#     ... in the South?
#     ... in the Midwest?
#     ... in the Southwest?
#     ... in the Northwest?
#     ... outside the contiguous US states?


In [21]:
# Loading the file data in

sales_df = pd.read_csv('sales.csv')
products_df = pd.read_csv('products.csv')
customers_df = pd.read_csv('customers.csv')
cities_df = pd.read_csv('cities.csv')
countries_df = pd.read_csv('countries.csv')


In [27]:
# Region mapping dictionary

region_mapping = {
    'Northeast': [
        'Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 
        'Vermont', 'New Jersey', 'New York', 'Pennsylvania'
    ],
    'Midwest': [
        'Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa',
        'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota'
    ],
    'South': [
        'Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 
        'South Carolina', 'Virginia', 'West Virginia', 'Alabama', 'Kentucky',
        'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas'
    ],
    'West': [
        'Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico',
        'Utah', 'Wyoming', 'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington'
    ]
}

non_contiguous = ['Alaska', 'Hawaii']

# Converting region_mapping to state-to-region format

state_to_region = {}
for region, states in region_mapping.items():
    for state in states:
        state_to_region[state] = region

In [23]:
# Calculating the actual price of each sale (accounting for quantity and discount)
sales_with_products = sales_df.merge(products_df[['ProductID', 'Price']], on='ProductID')
sales_with_products['actual_price'] = (
    sales_with_products['Price'] * 
    sales_with_products['Quantity'] * 
    (1 - sales_with_products['Discount'])
)

In [24]:
# Calculating cart values by grouping by transaction
cart_values = (sales_with_products
    .groupby(['TransactionNumber', 'CustomerID'])['actual_price']
    .sum()
    .reset_index())


In [25]:
# Getting customer location information
customer_locations = (customers_df
    .merge(cities_df, on='CityID')
    .merge(countries_df, on='CountryID')
)

print(customer_locations.columns)
print(customer_locations.head())

Index(['CustomerID', 'FirstName', 'MiddleInitial', 'LastName', 'CityID',
       'Address', 'CityName', 'Zipcode', 'CountryID', 'CountryName',
       'CountryCode'],
      dtype='object')
   CustomerID FirstName MiddleInitial LastName  CityID  \
0           1  Stefanie             Y     Frye      79   
1           2     Sandy             T    Kirby      96   
2           3       Lee             T    Zhang      55   
3           4    Regina             S    Avery      40   
4           5    Daniel             S   Mccann       2   

                        Address    CityName  Zipcode  CountryID  \
0                 97 Oak Avenue    Oklahoma    40472         32   
1        52 White First Freeway  Pittsburgh    14257         32   
2       921 White Fabien Avenue     Houston    95800         32   
3                 75 Old Avenue   Cleveland    51352         32   
4  283 South Green Hague Avenue     Buffalo    17420         32   

     CountryName CountryCode  
0  United States          AR  

In [26]:
# Isolating US customers
